In [11]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [37]:
import pandas as pd
dftre=pd.read_csv('/content/gdrive/My Drive/Eleições Municipais 2020/NIC_6662021_DANIEL_NEVES_DOS_SANTOS_LOCAIS_2021_2.csv',encoding='latin-1', sep=';')
dftre.head()

,CANDIDATO,NOME_URNA,NR_CANDIDATO,NOM_LOCALIDADE,ELEICAO,NM_CARGO,PARTIDO,COLIGA_PART,TIPO_VOTACAO,NRSECAO,LOCAL_VOTACAO,ENDERECO,QTVOTOS
0,JOICE CRISTINA HASSELMANN,JOICE CRISTINA HASSELMANN,17.0,SÃO PAULO,ELEIÇÕES 2020,Prefeito,Partido Social Liberal - PSL,"SP MERECE MAIS - DC, PSL",NORMAL,95,COLÉGIO JOÃO E RAPHAELA PASSALACQUA,"RUA JOÃO PASSALACQUA, 207 - BELA VISTA",6
1,JOICE CRISTINA HASSELMANN,JOICE CRISTINA HASSELMANN,17.0,SÃO PAULO,ELEIÇÕES 2020,Prefeito,Partido Social Liberal - PSL,"SP MERECE MAIS - DC, PSL",NORMAL,218,EE. PROFESSORA MARINA CINTRA,"RUA DA CONSOLAÇÃO, 1289 - CONSOLACAO",1
2,JOICE CRISTINA HASSELMANN,JOICE CRISTINA HASSELMANN,17.0,SÃO PAULO,ELEIÇÕES 2020,Prefeito,Partido Social Liberal - PSL,"SP MERECE MAIS - DC, PSL",NORMAL,234,EE. CAETANO DE CAMPOS,"RUA JOÃO GUIMARÃES ROSA, 111 - CONSOLACAO",1
3,JOICE CRISTINA HASSELMANN,JOICE CRISTINA HASSELMANN,17.0,SÃO PAULO,ELEIÇÕES 2020,Prefeito,Partido Social Liberal - PSL,"SP MERECE MAIS - DC, PSL",NORMAL,454,UNIV. CRUZEIRO DO SUL - UNICSUL,"RUA GALVÃO BUENO, 868 - LIBERDADE",7
4,JOICE CRISTINA HASSELMANN,JOICE CRISTINA HASSELMANN,17.0,SÃO PAULO,ELEIÇÕES 2020,Prefeito,Partido Social Liberal - PSL,"SP MERECE MAIS - DC, PSL",NORMAL,519,ANHEMBI MORUMBI - LIB.BADARÓ (ANHANGABAÚ),"RUA LÍBERO BADARÓ, 487 - CENTRO",3


In [38]:
dftre.info()
#Início da Análise Exploratória

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2927870 entries, 0 to 2927869
Data columns (total 13 columns):
 #   Column          Dtype  
---  ------          -----  
 0   CANDIDATO       object 
 1   NOME_URNA       object 
 2   NR_CANDIDATO    float64
 3   NOM_LOCALIDADE  object 
 4   ELEICAO         object 
 5   NM_CARGO        object 
 6   PARTIDO         object 
 7   COLIGA_PART     object 
 8   TIPO_VOTACAO    object 
 9   NRSECAO         int64  
 10  LOCAL_VOTACAO   object 
 11  ENDERECO        object 
 12  QTVOTOS         int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 290.4+ MB


In [39]:
#Optei por deixar o número do candidato e número da seção como string (e não float e inteiro respectivamente) já que não farei nenhuma operação matemática com eles
dftre=dftre.astype({'NR_CANDIDATO':'str','NRSECAO':'str'})
#Quis deixar visualmente mais agradável a apresentação do dataframe, isto é, sem o ".0" residual da transformação. I removed 2 last caracters from a pandas series.
#Não executar mais que 1 vez senão começa a mutilar a coluna NR_CANDIDATO
dftre['NR_CANDIDATO']=dftre['NR_CANDIDATO'].map(lambda x: str(x)[:-2])
dftre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2927870 entries, 0 to 2927869
Data columns (total 13 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   CANDIDATO       object
 1   NOME_URNA       object
 2   NR_CANDIDATO    object
 3   NOM_LOCALIDADE  object
 4   ELEICAO         object
 5   NM_CARGO        object
 6   PARTIDO         object
 7   COLIGA_PART     object
 8   TIPO_VOTACAO    object
 9   NRSECAO         object
 10  LOCAL_VOTACAO   object
 11  ENDERECO        object
 12  QTVOTOS         int64 
dtypes: int64(1), object(12)
memory usage: 290.4+ MB


In [59]:
#Isolei todos os endereços únicos para de alguma maneira enriquecer com a latitude e longitude
dfenderecos = dftre[['ENDERECO','NOM_LOCALIDADE']]
dfenderecos=dfenderecos.drop_duplicates('ENDERECO')
dfenderecos.tail()
#dfenderecos.to_excel('sp-diadema.xlsx')

,ENDERECO,NOM_LOCALIDADE
33947,"AV. MIGUEL IGNACIO CURI, 1311 - ITAQUERA",SÃO PAULO
34873,"RODOVIA RAPOSO TAVARES, KM 19,5 - JARDIM ARPOADOR",SÃO PAULO
36088,"AV. NAÇÕES UNIDAS, 1233 - VILA LEOPOLDINA",SÃO PAULO
44919,"AV. NAÇÕES UNIDAS, 1405 - VILA LEOPOLDINA",SÃO PAULO
704860,"RUA CARAMURU, 1255 - VILA CONCEIÇÃO",DIADEMA


In [67]:
dfenderecos['ENDERECO'].str.split(' - ',expand=True)
#Apenas experimentando, mas alguns poucos endereços tem mais de um " - " e aí não rolou uma separação perfeita

,0,1,2
0,"RUA JOÃO PASSALACQUA, 207",BELA VISTA,None
1,"RUA DA CONSOLAÇÃO, 1289",CONSOLACAO,None
2,"RUA JOÃO GUIMARÃES ROSA, 111",CONSOLACAO,None
3,"RUA GALVÃO BUENO, 868",LIBERDADE,None
4,"RUA LÍBERO BADARÓ, 487",CENTRO,None
...,...,...,...
33947,"AV. MIGUEL IGNACIO CURI, 1311",ITAQUERA,None
34873,"RODOVIA RAPOSO TAVARES, KM 19,5",JARDIM ARPOADOR,None
36088,"AV. NAÇÕES UNIDAS, 1233",VILA LEOPOLDINA,None
44919,"AV. NAÇÕES UNIDAS, 1405",VILA LEOPOLDINA,None


In [72]:
dfenderecos['ENDERECO'].str.split(' - ',expand=True)[2].value_counts()
#Ainda que as 9 exceções dê pra fazer o ajuste na mão...

JARDIM SANTA BÁRBARA    5
SAO JOAO CLIMACO        1
VILA OLÍMPIA            1
PARELHEIROS             1
JARDIM AMERICA          1
Name: 2, dtype: int64